# STAT 547E — Assignment 1: Stabilised MCMC
**Due: March 11, 2026**

Submit a single PDF containing all derivations, code, and figures.
Include well-commented code and clearly labelled plots.

---
**Setup.**
Let $\pi(x) = \gamma(x)/Z$ be a target on $\mathbb{X}$, $\eta$ a reference distribution,
$$w(x) = \frac{\gamma(x)}{\eta(x)}, \qquad \alpha(x,y) = \min\!\left\{1,\,\frac{w(y)}{w(x)}\right\}.$$
The **Stabilised MCMC** algorithm at each step $t$:
1. Draw $X_t' \sim K(X_{t-1}, \cdot)$ and $Y_t' \sim \eta$ independently.
2. With probability $\alpha(X_t', Y_t')$ set $(X_t, Y_t) = (Y_t', X_t')$; otherwise $(X_t, Y_t) = (X_t', Y_t')$.

The empirical swap rejection rate and IS estimate of $Z$ are
$$\hat{r}_T = 1 - \frac{1}{T}\sum_{t=1}^T \alpha_t, \qquad \hat{Z}_T = \frac{1}{T}\sum_{t=1}^T w(Y_t).$$

---
## Setup

In [ ]:
using Random, Statistics, LinearAlgebra
using Distributions
using Plots, LaTeXStrings

Random.seed!(42)
default(fontfamily = "Computer Modern", framestyle = :box,
        label = nothing, grid = true, gridalpha = 0.3,
        linewidth = 2, dpi = 150)

logsumexp(v) = (m = maximum(v); m + log(sum(exp.(v .- m))))

---
## Problem 3 — Gaussian case

Suppose $\pi = \mathcal{N}(\mu, \sigma^2)$, $\eta = \mathcal{N}(0, \sigma^2)$, and let $\rho = \mu/\sigma$.

### 3(a)

Show that $\hat{r}_T$ converges a.s. to
$$r(\rho) = 2\Phi\!\left(\frac{\rho}{\sqrt{2}}\right) - 1,$$
where $\Phi$ is the standard normal CDF.
Plot $r(\rho)$ and the expected waiting time $r/(1-r)$ as functions of $\rho \in [0, 5]$.
How do these scale as $\rho \to 0$ and $\rho \to \infty$?

*Hint: the TV between two Gaussians with the same covariance $\Sigma$ and means $a$, $b$ is*
$$\|\mathcal{N}(a,\Sigma) - \mathcal{N}(b,\Sigma)\|_{\mathrm{TV}} = 2\Phi\!\left(\tfrac{1}{2}\|a-b\|_{\Sigma^{-1}}\right) - 1, \qquad \|a-b\|_{\Sigma^{-1}} := \sqrt{(a-b)^\top \Sigma^{-1}(a-b)}.$$

**Your derivation here.**

In [ ]:
# Your code here
# Define r_fn(ρ) and wait_fn(ρ), then plot both

### 3(b)

Show that $\hat{Z}_T$ is unbiased with variance
$$\mathbb{E}[\hat{Z}_T] = 1, \qquad \mathbb{V}[\hat{Z}_T] = \frac{e^{\rho^2} - 1}{T}.$$
Verify numerically by running 200 independent replicates of the algorithm for $\rho \in \{0.5, 1, 2, 3\}$
and comparing the empirical variance $\times T$ to the theoretical value.

**Your derivation here.**

In [ ]:
# Your code here

### 3(c)

Compare how the expected waiting time $r(\rho)/(1-r(\rho))$ and the variance $\mathbb{V}[\hat{Z}_T]$
scale as $\rho \to 0$ and $\rho \to \infty$. Do they deteriorate at the same rate?

**Your answer here.**

In [ ]:
# Your code here (plots of waiting time and IS variance on separate panels)

---
## Problem 4 — Classical MCMC on GMM-40

**Target:**
$$\pi(x) = \frac{1}{40}\sum_{k=1}^{40}\mathcal{N}(x \mid \mu_k,\, 0.1\, I_2),$$
with means $\mu_k$ on a $5 \times 8$ grid of spacing $\Delta = 3$:
$\mu_k \in \{0,3,6,9,12\} \times \{0,3,\ldots,21\}$.

Implement and run for $T = 10{,}000$ iterations:
- **(i)** RWM with $\sigma_\mathrm{prop} \in \{0.3,\, 1,\, 3\}$.
- **(ii)** MALA with $\epsilon \in \{0.05,\, 0.1,\, 0.5\}$.

For each: report a scatter plot of samples on contours of $\pi$, a traceplot of $x_1$,
and the ESS of $x_1$. Comment on the effect of the proposal scale.

In [ ]:
# GMM-40 specification
const σ²     = 0.1
const Δ      = 3
const μ_grid = [[i*Δ, j*Δ] for i in 0:4 for j in 0:7]   # 40 means
const K      = length(μ_grid)

function log_gamma(x::Vector)
    # TODO: implement log unnormalised density via log-sum-exp
end

function grad_log_gamma(x::Vector)
    # TODO: implement gradient of log γ(x)
end

# Contour grid for visualisation
xg = range(-2, 14, length=150)
yg = range(-2, 23, length=150)
Z_contour = [exp(log_gamma([x, y])) for y in yg, x in xg];

In [ ]:
# ESS via integrated autocorrelation time
function ess_1d(x::Vector)
    n  = length(x)
    xc = x .- mean(x)
    v  = var(x)
    v < 1e-10 && return 1.0
    τ = 1.0
    for k in 1:min(n÷2, 2000)
        ρk = dot(xc[1:n-k], xc[k+1:n]) / ((n-k) * v)
        ρk < 0 && break
        τ += 2ρk
    end
    return n / τ
end

In [ ]:
# TODO: implement rwm_chain(x0, σ_prop, T; rng)
# Returns (chain::Matrix{Float64}[2×T], acceptance_rate::Float64)

In [ ]:
# TODO: implement mala_chain(x0, ε, T; rng)
# Proposal: x' = x + (ε²/2)∇log γ(x) + ε ξ,  ξ ~ N(0, I₂)
# Remember to include the MH correction for the asymmetric proposal

In [ ]:
T  = 10_000
x0 = μ_grid[1] .+ 0.1*randn(2)

# TODO: run sweeps over σ_props = [0.3, 1.0, 3.0] and εs = [0.05, 0.1, 0.5]
# Report acceptance rates and ESS(x₁)

In [ ]:
# TODO: scatter plots on contours + traceplots for each tuning value
# Arrange as (scatter row, traceplot row) × 3 columns for each algorithm

**Comments on proposal scale:**

---
## Problem 5 — Stabilised MCMC on GMM-40

Apply the Stabilised MCMC algorithm to GMM-40 using both RWM ($\sigma_\mathrm{prop}=1$)
and MALA ($\epsilon=0.1$) as local kernels.

In [ ]:
# TODO: implement stabilised_mcmc(σ_η, T; kernel, σ_prop, ε, rng)
# Returns (Xs::Matrix{Float64}[2×T], r_hat::Float64)
#
# Each iteration:
#   Exploration: advance X via K (RWM or MALA); draw Y' ~ η fresh
#   Swap: with prob min(1, w(Y')/w(X)) exchange X and Y'

### 5(a)

Run with $\eta = \mathcal{N}(0, \sigma_\eta^2 I_2)$, $\sigma_\eta = 10$, $T = 10{,}000$.

Report for both RWM and MALA:
- Scatter plots of $(X_t)$ overlaid on contours of $\pi$, side by side with the classical sampler.
- Traceplots of $x_1$, side by side with the classical sampler.
- Final ESS of $x_1$ and empirical rejection rate $\hat{r}_T$.

In [ ]:
σ_η = 10.0
T   = 10_000

# TODO: run classical and stabilised versions; print ESS and rejection rate table

In [ ]:
# TODO: 2×2 figure — top row: scatter (classical | stabilised)
#                  — bottom row: traceplot (classical | stabilised)
# Produce one figure for RWM, one for MALA

### 5(b)

Repeat for $\sigma_\eta \in \{2^{-5}, 2^{-4}, \ldots, 2^5\}$.
Plot the final ESS and $\hat{r}_T$ as functions of $\sigma_\eta$.
Discuss how the reference distribution influences performance.

In [ ]:
σ_η_vals = [2.0^k for k in -5:5]

# TODO: sweep over σ_η_vals; collect ESS and r_hat for each

In [ ]:
# TODO: side-by-side plots of ESS and r_hat vs σ_η (log₂ x-axis)

**Discussion:**